In [4]:
import torchvision
import torch
from PIL import Image
from torchvision.models.detection.mask_rcnn import MaskRCNN
from torchvision.transforms import ToTensor
from torchvision.datasets import DatasetFolder
from torch.utils.data import DataLoader

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, root, transforms):
        self.root = root
        self.transforms = transforms
        self.dataset = DatasetFolder(root, loader=torchvision.datasets.folder.default_loader, extensions=('.jpg', '.png', '.jpeg'))

    def __getitem__(self, idx):
        img_path, target = self.dataset.samples[idx]
        image = Image.open(img_path).convert("RGB")
        image = self.transforms(image)
        return image, target

    def __len__(self):
        return len(self.dataset.samples)
train_folder = "/Users/vedantpadole/Desktop/Research/American sign language.v2i.yolov5pytorch/train/images"
test_folder = "/Users/vedantpadole/Desktop/Research/American sign language.v2i.yolov5pytorch/test/images"
validate_folder = "/Users/vedantpadole/Desktop/Research/American sign language.v2i.yolov5pytorch/valid/images"
data_yaml = "/Users/vedantpadole/Desktop/Research/American sign language.v2i.yolov5pytorch/data.yaml"


import yaml
with open(data_yaml, 'r') as f:
    config = yaml.safe_load(f)
transform = ToTensor()
train_dataset = CustomDataset(train_folder, transform)
test_dataset = CustomDataset(test_folder, transform)
validate_dataset = CustomDataset(validate_folder, transform)
train_dataloader = DataLoader(train_dataset, batch_size=2, shuffle=True, num_workers=4)
test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=4)
validate_dataloader = DataLoader(validate_dataset, batch_size=1, shuffle=False, num_workers=4)
model = MaskRCNN(num_classes=len(config['names']), pretrained=True)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    for images, targets in train_dataloader:
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()
    lr_scheduler.step()
    model.eval()
    with torch.no_grad():
        for images, targets in validate_dataloader:
            images = list(image.to(device) for image in images)
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
            output = model(images)
torch.save(model.state_dict(), 'mask_rcnn_trained.pth')


FileNotFoundError: Couldn't find any class folder in /Users/vedantpadole/Desktop/Research/American sign language.v2i.yolov5pytorch/train/images.